In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('payscale-all-majors-colleges.csv')

In [3]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [4]:
pysqldf("SELECT * FROM df ORDER BY mid DESC LIMIT 100")

,Unnamed: 0,index,school,early,mid,major
0,2750,0,Harvard University,96600,205600,math
1,525,0,Harvey Mudd College,117700,189200,computer-science
2,526,1,Stanford University,119600,183100,computer-science
3,527,2,University of California-Berkeley,115700,181100,computer-science
4,528,3,Harvard University,103900,179400,computer-science
...,...,...,...,...,...,...
95,574,24,University of California-Davis,98300,150000,computer-science
96,575,0,Trinity College,89800,150000,computer-science
97,576,1,LeTourneau University,78100,149800,computer-science
98,1139,14,United States Air Force Academy,76200,149700,social-sciences


As we analyze this data, keep in mind that some majors are more common than others. For example, there are 600 computer science programs represented and only 114 math programs, so if math and computer science are equally distributed, we'd expect to see over five times as many CS programs as math programs in the top 100. 

To factor this into comparison, I created an adjustment - in later comparisons we can divide by this adjustment factor to include relative frequency in the overall dataset into our rankings. 

In [5]:
pysqldf("SELECT COUNT(*) * 1 FROM df")

,COUNT(*) * 1
0,4425


In [6]:
major_counts = pysqldf("""
SELECT major, COUNT(major), COUNT(major) * 1.0 / (SELECT COUNT(*) * 1.0 FROM df) AS adj
FROM df
GROUP BY major
""")

major_counts

,major,COUNT(major),adj
0,art,400,0.090395
1,communication,425,0.096045
2,computer-science,600,0.135593
3,engineering,400,0.090395
4,health-science,650,0.146893
5,humanities,525,0.118644
6,math,100,0.022599
7,physical-and-life-sciences,500,0.112994
8,social-sciences,825,0.186441


Query the top 100 majors by mid career pay

In [7]:
top_100_late = pysqldf("SELECT * FROM df ORDER BY mid DESC LIMIT 100")

Frequency of each field in the top 100

In [8]:
pysqldf("""
SELECT t.major, COUNT(t.major)
FROM top_100_late t
GROUP BY t.major
ORDER BY COUNT(t.major) DESC
""")

,major,COUNT(t.major)
0,computer-science,52
1,engineering,23
2,social-sciences,15
3,math,8
4,physical-and-life-sciences,1
5,health-science,1


Adjusted for over/under representation of each major in the data set

In [9]:
pysqldf("""
SELECT t.major, mc.adj, COUNT(t.major), COUNT(t.major) / mc.adj AS adjusted
FROM top_100_late t
JOIN
major_counts mc
ON t.major = mc.major
GROUP BY t.major
ORDER BY COUNT(t.major) DESC
""")

,major,adj,COUNT(t.major),adjusted
0,computer-science,0.135593,52,383.500000
1,engineering,0.090395,23,254.437500
2,social-sciences,0.186441,15,80.454545
3,math,0.022599,8,354.000000
4,physical-and-life-sciences,0.112994,1,8.850000
5,health-science,0.146893,1,6.807692


In [10]:
pd.set_option('display.max_rows', None)

Query the top 10 majors for each sub field ordered by late career pay. 

Keep in mind this can be a tricky query to interpret. Remember that this is the top 10 for each department. It's easy to mentally slip into thinking you're looking at the top 110 overall. 

In [11]:
q = """
SELECT r.*
FROM
(
    SELECT
        r.*,
        ROW_NUMBER() OVER(PARTITION BY r.major
                          ORDER BY r.mid DESC) rn
    FROM df r
) r
WHERE r.rn <= 10
ORDER BY mid DESC
"""

pysqldf(q)

,Unnamed: 0,index,school,early,mid,major,rn
0,2750,0,Harvard University,96600,205600,math,1
1,525,0,Harvey Mudd College,117700,189200,computer-science,1
2,526,1,Stanford University,119600,183100,computer-science,2
3,527,2,University of California-Berkeley,115700,181100,computer-science,3
4,528,3,Harvard University,103900,179400,computer-science,4
5,529,4,Carnegie Mellon University,109200,178300,computer-science,5
6,530,5,Massachusetts Institute of Technology,109300,177200,computer-science,6
7,531,6,Princeton University,115100,177100,computer-science,7
8,532,7,Columbia University in the City of New York,104300,175400,computer-science,8
9,533,8,Dartmouth College,113300,174900,computer-science,9


In [12]:
a = 0.1

df_late_t10p = (df.groupby('major',group_keys=False)
        .apply(lambda x: x.nlargest(int(len(x) * a), 'mid')))

In [20]:
df_late_t10p.reset_index().sort_values("mid", ascending=False)

,level_0,Unnamed: 0,index,school,early,mid,major
299,2750,2750,0,Harvard University,96600,205600,math
82,525,525,0,Harvey Mudd College,117700,189200,computer-science
83,526,526,1,Stanford University,119600,183100,computer-science
84,527,527,2,University of California-Berkeley,115700,181100,computer-science
85,528,528,3,Harvard University,103900,179400,computer-science
86,529,529,4,Carnegie Mellon University,109200,178300,computer-science
87,530,530,5,Massachusetts Institute of Technology,109300,177200,computer-science
88,531,531,6,Princeton University,115100,177100,computer-science
89,532,532,7,Columbia University in the City of New York,104300,175400,computer-science
90,533,533,8,Dartmouth College,113300,174900,computer-science
